In [ ]:
# создание таблицы с пропусками

def nan_info(df):
    global null_view
    try:
        null_view = df[[col for col in df.columns if df[col].isna().sum() > 0]].isna().sum().sort_values(ascending = True)
        null_view = pd.DataFrame(null_view, columns=['NANs'])
        null_view['PERCENT'] = null_view.NANs.apply(lambda x: round((x/len(df))*100, 2))
        null_view['TYPE'] = df.dtypes
    except:
        print('ERROR')
        return null_view
    return null_view

NaN_info(df)

In [ ]:
def moustache(s):
    # Принемает series
    # Возращает значения усов нормального распределения
    q75,q25 = np.percentile(s,[75,25])
    iqr = q75 - q25
    return q25 - 1.5*iqr, q75 + 1.5*iqr

def data_cleanup_table(df):
    # Возвращает таблицу для EDA с анализом признаков
    
    # Создание таблицы
    columns = ['Признак', 'Тип_данных', 'Пропущенных_значений', 
           'Выбросов', 'Ошибочных_значений', 'Управление', 'Комментарий']
    table = pd.DataFrame(columns=columns)
    table['Признак'] = df.columns
    df_len = len(df)
    print('Создание таблицы...')
    #
    for sign in df.columns:
        table.loc[table['Признак'] == sign, 'Тип_данных'] = df[sign].dtype
    
    
    # Анализ пропущенных значений
    for sign in tqdm(df.columns, desc='Анализ пропущенных значений\t'):
        table.loc[table['Признак'] == sign, 'Пропущенных_значений'] = df[sign].isna().sum()
        if df[sign].isna().sum() / df_len > 0.2:
            table.loc[table['Признак'] == sign, 'Управление'] = 'Удалить'
            table.loc[table['Признак'] == sign, 'Комментарий'] = 'Пропусков больше 20% '
            
    
    # Анализ выбрасов
    for sign in tqdm(df.columns, desc='Анализ выбросов\t\t\t'):
        if df[sign].dtype == 'object':
            table.loc[table['Признак'] == sign, 'Выбросов'] = -1
        else:
            tmp = moustache(df[sign]) # возращает границы усов (а, б)
            table.loc[table['Признак'] == sign, 'Выбросов'] = df[(df[sign] < tmp[0]) |
                                                                            (df[sign] > tmp[1])][sign].count()      
    
    return table

In [41]:
# ПРИМЕР:    НЕ ЗАПУСКАТЬ!
table = data_cleanup_table(df_train)
table.head()

Создание таблицы...


Анализ выбросов			: 100%|██████████| 291/291 [00:02<00:00, 106.60it/s]


,Признак,Тип_данных,Пропущенных_значений,Выбросов,Ошибочных_значений,Управление,Комментарий
0,id,int64,0,0,NaN,NaN,NaN
1,timestamp,object,0,-1,NaN,NaN,NaN
2,full_sq,int64,0,963,NaN,NaN,NaN
3,life_sq,float64,6383,0,NaN,Удалить,Пропусков больше 20%
4,floor,float64,167,0,NaN,NaN,NaN


---
---

In [4]:
# Приметивна функция преоброзование даты, надо доработать\

# Обработка дат ДОП!
# df['col'] = pd.to_datetime(df['col']) – меняет тип на datetime64[ns]
# df['col'].dt.day – вытаскивает из столбца col только значение дня
# df['col'].dt.month – только месяц
# df['col'].dt.year – только год


def split_datetime(df):
    day_list = ('Понедельник', 'Вторник', 'Среда', 'Четверг', 'Пятница', 'Суббота', 'Воскресенье')
    df_datetime = df['datetime'].apply(lambda x: dt.strptime(x, '%Y-%m-%d %H:%M:%S'))
    
    df['date'] = df_datetime.apply(lambda x: str(x.date()))
    df['year'] = df_datetime.apply(lambda x: str(x.date()).split('-')[0])
    df['month'] = df_datetime.apply(lambda x: str(x.date()).split('-')[1])
    df['day'] = df_datetime.apply(lambda x: str(x.date()).split('-')[2])
    df['week_number'] = df_datetime.apply(lambda x: x.isocalendar()[1])
    df['weekday'] = df_datetime.apply(lambda x: day_list[x.isocalendar()[2] - 1])
    df['time'] = df_datetime.apply(lambda x: str(x.time()))
    df['hour'] = df_datetime.apply(lambda x: str(x.hour))
    df['min'] = df_datetime.apply(lambda x: str(x.minute))
    df['sec'] = df_datetime.apply(lambda x: str(x.second))
    df['sec_of_day'] = df_datetime.apply(lambda x: str(x.hour*3600 + x.minute*60 + x.second))
    
    return df

df = split_datetime(df)
df.head()
nik_name = 'Igarg'
df.to_excel('../datasets/Даты '+nik_name+'.xlsx', index=False)
df

,datetime,date,year,month,day,week_number,weekday,time,hour,min,sec,sec_of_day
0,2021-01-26 17:22:41,2021-01-26,2021,01,26,4,Вторник,17:22:41,17,22,41,62561
1,2013-04-18 14:44:04,2013-04-18,2013,04,18,16,Четверг,14:44:04,14,44,4,53044
2,2017-11-12 21:51:06,2017-11-12,2017,11,12,45,Воскресенье,21:51:06,21,51,6,78666
3,2001-09-14 22:48:09,2001-09-14,2001,09,14,37,Пятница,22:48:09,22,48,9,82089
4,2012-12-03 18:05:30,2012-12-03,2012,12,03,49,Понедельник,18:05:30,18,5,30,65130
...,...,...,...,...,...,...,...,...,...,...,...,...
95,2002-04-06 14:45:02,2002-04-06,2002,04,06,14,Суббота,14:45:02,14,45,2,53102
96,2010-01-17 19:29:42,2010-01-17,2010,01,17,2,Воскресенье,19:29:42,19,29,42,70182
97,2016-09-20 13:10:02,2016-09-20,2016,09,20,38,Вторник,13:10:02,13,10,2,47402
98,2019-11-27 15:05:00,2019-11-27,2019,11,27,48,Среда,15:05:00,15,5,0,54300


Одним из способов улучшения классификатора является увеличение порядка пространства признаков, например, если на плоскости достаточно тяжело провести линию, разделяющую классы, то в трехмерном пространстве уже можно найти плоскость, которая сделает это лучше.

Все моменты времени необходимо преобразовывать через синус и косинус, формируя тем самым вместо одного два признака. Это становится тем более важным, если у нас вместо одного временного периода имеется несколько и существует возможность периодических колебаний.

Преобразования производятся по следующим формулам:
$$z_{sin} = sin\left(\frac{2\cdot\mathrm{\pi} \cdot \mathrm{x}}{ \mathrm{K} }\right)$$ 
$$z_{cos} = cos\left(\frac{2\cdot\mathrm{\pi} \cdot \mathrm{x}}{ \mathrm{K} }\right)$$ 

Здесь х - это значение момента времени, а К - значение временного периода в единицах измерения моментов времени.
Например: 
- если моменты времени измеряются в секундах, а временной период равен суткам, то К=86400;
- если моменты времени измеряются в днях, а временной период равен неделе, то К=7.

Проведем преобразование времени поступления заказа:

In [125]:
df['Order_sin'] = np.sin(df.Order_time_sec*math.pi/43200)
df['Order_cos'] = np.cos(df.Order_time_sec*math.pi/43200)
df

,Order_time,Delivery_time,Office,Order_time_sec,Order_time_h,Order_time_sec_std_1,Delivery_time_std_1,y_pred_1,Correct_1,Order_time_sec_delta,Order_time_sec_delta_std_delta,Delivery_time_std_delta,y_pred_delta,Correct_delta,Order_sin,Order_cos
0,0:03:35,12,0,215,0,-2.670995,1.403727,1,False,5877,-2.456039,1.403727,1,False,0.015635,0.999878
1,0:07:14,6,1,434,0,-2.659361,-0.473870,1,True,6096,-2.444885,-0.473870,1,True,0.031556,0.999502
2,0:11:30,9,1,690,0,-2.645761,0.464929,1,True,6352,-2.431846,0.464929,1,True,0.050157,0.998741
3,0:13:48,12,0,828,0,-2.638430,1.403727,1,False,6490,-2.424817,1.403727,1,False,0.060177,0.998188
4,0:14:36,2,1,876,0,-2.635880,-1.725600,1,True,6538,-2.422372,-1.725600,1,True,0.063661,0.997972
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,23:39:14,3,1,85154,23,1.841378,-1.412668,0,False,4416,-2.530453,-1.412668,1,True,-0.090488,0.995898
276,23:40:55,9,1,85255,23,1.846744,0.464929,0,False,4517,-2.525309,0.464929,1,True,-0.083171,0.996535
277,23:45:19,6,1,85519,23,1.860769,-0.473870,0,False,4781,-2.511863,-0.473870,1,True,-0.064024,0.997948
278,23:47:07,4,1,85627,23,1.866506,-1.099735,0,False,4889,-2.506362,-1.099735,1,True,-0.056185,0.998420


## Функция для заполнения NaN, посредством lightgbm()

#### Пользоваться осторожно, нет проверки на качество предсказания

In [ ]:
# ФУНКЦИЯ РАБОТАЕТ НЕКОРЕКТНО!!!!!!
# НИЖЕ РАБОЧИЯ ВЕРСИЯ!!!!!


def encoding(x, y, LE):
    for col in x.columns:
        if x[col].dtype == 'object':
            x[col].replace(np.nan, '0', inplace=True)
            x.loc[:, col] = LE.fit_transform(x.loc[:, col])
    y.loc[~y.isna()] = LE.fit_transform(y.loc[~y.isna()])

    return LE

def get_variable(df, col_pred):
    # Делит данные по индексу которые нужно предсказать
    y_now = df[col_pred].copy()
    x_now = df[set(df.columns) - set([col_pred])].copy() # Ошибка здесь [col_pred]
    
    # Декодируем
    LE = encoding(x_now, y_now, LabelEncoder())
    
    index_train = y_now[~y_now.isna()].index
    index_target = y_now[y_now.isna()].index
    
    x_train = x_now.drop(index=index_target)
    y_train = y_now.drop(index=index_target)
    x_now.drop(index=index_train, inplace=True)
    y_now.drop(index=index_train, inplace=True)
    return x_train, y_train.astype('int32'), x_now, y_now, LE

# требует доработки с разделением на train и test и выводом метрики качества
def predict_regressor(x_train, y_train, x_target):
    lgbm = lgb.LGBMRegressor()
    lgbm.fit(x_train, y_train)
    pred_miss = lgbm.predict(x_target)
    print('Start: predict_regressor')
    print('-------------------------------')
    return pred_miss

# требует доработки с разделением на train и test и выводом метрики качества
def predict_classifier(x_train, y_train, x_target):
    lgbm = lgb.LGBMClassifier()
    lgbm.fit(x_train, y_train)
    pred_miss = lgbm.predict(x_target)
    print('Start: predict_classifier')
    return pred_miss

def nan_predict(df_start):
    # Возращает новый dataframe с заполнеными NaN 
    import lightgbm as lgb
    from sklearn.preprocessing import LabelEncoder
    
    df = df_start.copy()
    nan_info(df) # Есть отдельно в этой шпаргалке
    
    predicted_features = null_view.index
    
    # Разделение на датафрейм
    for col_pred in predicted_features: 
        print(f"feature: {col_pred}, missing values: {null_view['PERCENT'][col_pred]}%, type: {null_view['TYPE'][col_pred]}")
        
        x_train, y_train, x_target, y_target, LE = get_variable(df, col_pred)
        
        if df[col_pred].dtype == 'object' or df[col_pred].dtype == 'bool':
            y_pred = predict_classifier(x_train, y_train, x_target)
            y_pred = LE.inverse_transform(y_pred)
        else:
            y_pred = predict_regressor(x_train, y_train, x_target)
        
        df.loc[y_target.index, y_target.name] = y_pred
        print(f"The first 100 predicted missing values: \n{y_pred[:100]}")
        print('-------------------------------\n')
        
        
    return df

In [1]:
# Recomendation

# cast the data types to the correct ones.
# remove irrelevant columns. (But it is not exactly)
# process those NaNs about which you know something for certain. !!!(try to skip this point)
# algorithm don't handle types: category, datetime64, timedelta[ns], maby bool(not tested)
# algorithm can handle features with 96.30% missing data, but can't handle 99.52% missing data. It DEPENDS on the number of raws.




def nan_predict(df_start,
                skip_features_with_missing_data_percentage=100, # If error, decrease to 99 or 90
                include_features_as_predictors_where_pec_miss_data_less=50,): 
    '''iteratively predict missing values in cells'''
    
    
    import pandas as pd
    import numpy as np

    # Disabling warnings
    import sys
    import warnings
    if not sys.warnoptions:
        warnings.simplefilter("ignore")


    import lightgbm as lgb
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import mean_absolute_error
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import f1_score
    from sklearn.preprocessing import LabelEncoder




    def NaN_info(df):
        global null_view
        try:
            null_view = df[[col for col in df.columns if df[col].isna().sum() > 0]].isna().sum().sort_values(ascending = True)
            null_view = pd.DataFrame(null_view, columns=['NANs'])
            null_view['PERCENT'] = null_view.NANs.apply(lambda x: round((x/len(df))*100, 2))
            null_view['TYPE'] = df.dtypes
        except:
            return null_view
        return null_view


    def encoding(work_predictors, df):
        for j in work_predictors:
            el_type = df[j].dtype
            if el_type == 'object':
                df[j].replace(np.nan, '0', inplace=True)
                labelencoder = LabelEncoder()
                df.loc[:, j] = labelencoder.fit_transform(df.loc[:, j])
        return df, work_predictors


    def predict_regressor(X, y, miss_df):
        global pred_miss
        lgbm = lgb.LGBMRegressor()
        lgbm.fit(X, y)
        pred_miss = list(lgbm.predict(miss_df))
        print('-------------------------------')
        print(f"The first 200 predicted missing values: \n{pred_miss[:200]}")
        return pred_miss


    def predict_classifier(X, y, miss_df):
        global pred_miss
        lgbm = lgb.LGBMClassifier()
        lgbm.fit(X, y)
        pred_miss = list(lgbm.predict(miss_df))
        print('-------------------------------')
        print(f"The first 200 predicted missing values: \n{pred_miss[:200]}")
        return pred_miss


    def imput_missing_value_to_main_df(df, miss_indeces, pred_miss, el):
        counter = 0
        for idx in miss_indeces:
            df.loc[idx, el] = pred_miss[counter]
            counter += 1
        return df



    
    df = df_start.copy()
    print(NaN_info(df))
    print('\n\n\n')

    delete_miss_features = list((null_view.loc[null_view['PERCENT'] > skip_features_with_missing_data_percentage]).index)
    print(f'Exclude from the prediction, because missing data more than \
    {skip_features_with_missing_data_percentage}% :\n{delete_miss_features}')
    print('')
    all_miss_features = list(null_view.index)

    for delete_feature in delete_miss_features:
        all_miss_features.remove(delete_feature)

    lot_of_miss_features = list((null_view.loc[null_view['PERCENT'] > include_features_as_predictors_where_pec_miss_data_less]).index)
    print(f'Unused as predictor, because missing data more than \
    {include_features_as_predictors_where_pec_miss_data_less}% :')
    print(lot_of_miss_features)
    print('')

    all_features = list(df.columns)
    now_predictors = list(set(all_features)-set(lot_of_miss_features))

    df_indeces = list(df.index)

    for el in all_miss_features:    
        work_predictors = list(set(now_predictors) - set([el]))

        # missing data (data for prediction)
        miss_indeces = list((df[pd.isnull(df[el])]).index)
        miss_df = df.iloc[miss_indeces][:]
        miss_df = miss_df[work_predictors]
        encoding(work_predictors, df=miss_df)

        # data without NaN rows (X data for train and evaluation of model)
        work_indeces = list(set(df_indeces) - set(miss_indeces))
        work_df = df.iloc[work_indeces][:] 
        encoding(work_predictors, df=work_df)

        X = work_df[work_predictors]
        y = work_df[el]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

        # Info
        feature_type = df[el].dtypes
        print('\n\n')
        perc = null_view['PERCENT'][el]
        print(f'feature: {el},   type: {feature_type},   missing values: {perc}%')    
        print(f'X.shape: {X.shape},   y.shape: {y.shape}')

        # Predictions
        if feature_type == 'object' or feature_type == 'bool':   # or feature_type == 'category' - error
            print('classifier:')
            predict_classifier(X, y, miss_df)
            imput_missing_value_to_main_df(df, miss_indeces, pred_miss, el)

        elif feature_type == 'float64' or feature_type == 'int64':
            print('regressor:')
            predict_regressor(X, y, miss_df)
            imput_missing_value_to_main_df(df, miss_indeces, pred_miss, el)

        else:
            print(f"unprocessed feature: {el} - {feature_type} type")


    print('\n\n\n')
    print(f'These features have not been processed, because missing data more than {skip_features_with_missing_data_percentage}%')
    print(NaN_info(df))
    print('\n\n\n')
    
    return df

In [ ]:
def cross_val_my_f(x, y, model):
    CV = ShuffleSplit(n_splits=5, test_size=20, random_state=21)
    cv_score = cross_val_score(model, x, y, cv=CV)
    
    return round(cv_score.mean(), 5)
    
def features_selection(x, y):
    # Функция вычисляет средний cv_score при удалении поочерёдно каждого признака
    # Если cv_score повышается, то признак попадает в список на удаление
    # Возвращает список рекомендованных к удалению признаков
    
    # Модель для анализа
    model = lgb.LGBMRegressor(random_state=21)
    
    features = list(x.columns)
    cv_score_best = cross_val_my_f(x, y, model)
    drop_features = [] 
    
    print(f'__Начало отбора__\n\nПри всех признаках cv_score = {cv_score_best}\n')
    
    # Пока есть признаки
    while len(features) > 1:
        scores_df = pd.DataFrame(index=features, columns=['cv_score'])
        print("Проверка cv_score при удаленных признаках")
        
        # Проверка cv_score при удалении каждого признака
        for col in tqdm(features):
            fit_features = list(set(features) - set([col]))
            scores_df.loc[col, 'cv_score'] = cross_val_my_f(x[fit_features], y, model)
        
        scores_df = scores_df.sort_values(by=['cv_score'], ascending=False)
        print('\n__Показатели cv_score при удаление признаков__')
        display(scores_df.head())
        
        # Если есть признак при котором cv_score повышается продолжаем цикл
        cv_score_next = scores_df.iloc[0,0]
        if cv_score_next > cv_score_best:
            cv_score_best = cv_score_next
            
            drop_features.append(scores_df.index[0])
            features = list(set(features) - set([scores_df.index[0]]))
            
            print('--------------------------------------------')
            print(f'    DROP   ====    {scores_df.index[0]}')
            print('--------------------------------------------')
            
        else:
            break
            
    print('--------------------------------------------')
    print("\n__Конец отбора__\n")
    print('--------------------------------------------')
    print(f"Рекомендовано удалить следующие признаки:\n {drop_features}")
    
    return drop_features

In [ ]:
def hyperparameter_tuning(x, y, algorithm, params):
    # подбор параметров для задач регрессии
    
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.1, random_state=21)
    CV = ShuffleSplit(n_splits=4, test_size=0.25, random_state=21)
    estimator = algorithm()
    
    randomized_model = RandomizedSearchCV(estimator,
                                         params,
                                         scoring='neg_mean_squared_error',
                                         cv=CV,
                                         n_iter=100,
                                         verbose=10000,
                                         n_jobs = -3) # для использованья многопоточности
    
    randomized_fited_best_model = randomized_model.fit(x_train, y_train)
    best_parameters = randomized_fited_best_model.best_params_
    pred_test = randomized_fited_best_model.predict(x_test)
    
    MAE = mean_absolute_error(y_test, pred_test)
    MSE = mean_squared_error(y_test, pred_test)
    model = algorithm(**best_parameters)
    cv_score = cross_val_score(model, x, y, cv=CV)
    
    print("\n" + '-'*100)
    print(f"MAE\t\t: {MAE}")
    print(f"MSE\t\t: {MSE}")
    print(f"RMSE\t\t: {MSE ** 0.5}")
    print(f"cv_score\t: {cv_score} - {cv_score.mean()} mean")
    print("\n" + '-'*100)
    print(f"best_parameters\t: {best_parameters}")
    
    return best_parameters

In [ ]:
def print_mean_error(y_main, y_pred):
    
        MAE = mean_absolute_error(y_main, y_pred)
        MSE = mean_squared_error(y_main, y_pred)

        print('-'*100)
        print(f"MAE:\t\t\t {MAE}")
        print(f"MSE:\t\t\t {MSE}")
        print(f"RMSE:\t\t\t {MSE ** 0.5}")
        print("\n" + '-'*100 + '\n')

def prediction(x, y, ML, params_dict, x_target):
    # ML - list моделей
    # param_dict = dict где key: имя модели в str, value: dict параметров модели 
    # return: dataframe из предсказаных y различными моделями 
    
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=21)
    
    # тестировка модели
    test_predict = pd.DataFrame()
    print('Проверка моделей' + '\n' + '-'*100)
    for model in ML:
        print(f"Выполняется работа:\t{model.__qualname__}")
        
        tmp_model = model(**params_dict[model.__qualname__])
        tmp_model.fit(x_train, y_train)
        test_predict[model.__qualname__] = tmp_model.predict(x_test)
        
        print_mean_error(y_test, test_predict[model.__qualname__])
    
    display(test_predict.head())
    print("Весь ансамбль")
    print_mean_error(y_test, test_predict.mean(axis=1))
    
    # предсказание
    predict_df = pd.DataFrame()
    print('ПРЕДСКАЗАНИЕ' + '\n' + '-'*100)
    for model in ML:
        print(f"Выполняется работа:\t{model.__qualname__}")
        
        tmp_model = model(**params_dict[model.__qualname__])
        tmp_model.fit(x, y)
        predict_df[model.__qualname__] = tmp_model.predict(x_target)
        
    print("\n" + '-'*100 + '\n')
    
    return predict_df